# This is parallel version of Merge sort 
### Merge sort is a population sort method that have time complexity is O($nlogn$) in any case.
but today, i decide to boost it, and check how fast merge sort can reach with parrallel version and maybe our mergesort can win quick sort, who knows. 

---

First of all we need to implement normal version of merge sort
### This is Merge Sort
Nothing to talk about this Algoritm
If you want do learn more about merge sort this [link](https://www.geeksforgeeks.org/merge-sort/) will usefull for you.


In [15]:
import numpy as np

def Merge(*args):
    res = []
    i = 0
    j = 0
    L,R = args[0] if len(args) == 1 else args
    while(i<len(L) and j<len(R)):
        if (L[i] < R[j]):
            res.append(L[i])
            i+=1
        else:
            res.append(R[j])
            j+=1
    if (i<len(L)):
        res = np.append(res, L[i:])
    if (j<len(R)):
        res = np.append(res, R[j:])
    return res
def MergeSort(A):
    if (len(A) <=1): 
        return A
    mid = int(len(A)/2)
    L = MergeSort(A[0:mid])
    R = MergeSort(A[mid:])
    return Merge(L,R)


### Time to mesuare time of sort algorithm
This function below will help you do that

In [16]:
import time
def Mesuare(arr , func):
    tic = time.time()
    arrS = func(arr)
    toc = time.time()
    MS_time = (toc-tic)*1000
    print(f'{func.__name__}, time used {MS_time} (ms) \n')
    return MS_time

# This time for parallel version

First of all we need to know basic idea of parallel version.

- All we know that merge sort first divide array into many small piece, sort it and assemble into an orderd array.

- But in normal merge sort we divide array until 1 element for each sub-array. we note that each sub array work independent. it's mean no order for each sub array in thís phase.

- So, why we dont use 1 process for each sub-array, sort its and assemble.

- But, number of process is limit and we denote that the time complexity n in O(nlogn) - a large part - come from merge phase not split phase.

- So, an optimize approach is split it into k such array sort each subarray and merge it after that. So far so good.

But what is k. We know that lower bound of k is 1 of course like the way we did before. First let's try with k is 2.

---
## Below implementation is merge sort parallel with two process run parallel in real time

Documentation about multiprocessing module you can find [here](https://docs.python.org/3/library/multiprocessing.html)

Wait.... What is MSPL, Ok..ok MSPL stand for Merge Sort Parallel Library (seems i got a library to solve this problem huh?), but, it just a custom module i wrote, you can find it in 
MSPL.py in same repo. Because jupyter notebook seem not work with multiprocessing module when you create a task for funcion in same notebook (i dont know why). So i create same 
function in MSPL.py file, build it, and import here. Its same with function i define above nothing is special here.

In [ ]:
import multiprocessing
def MergeSortParallel(A):
    if (len(A) <=1): 
        return A
    mid = int(len(A)/2)
    with multiprocessing.Pool(2) as pool:
        L = pool.apply_async(MergeSortParallel, [A[0:mid]])
        R = pool.apply_async(MergeSortParallel, [A[mid:]])
        L = L.get()
        R = R.get()
    return Merge(L,R)

In [17]:
import multiprocessing
import MSPL
def MergeSortParallelVer0(A):
    if (len(A) <=1): 
        return A
    mid = int(len(A)/2)
    with multiprocessing.Pool(2) as pool:
        L = pool.apply_async(MSPL.MergeSort, [A[0:mid]])
        R = pool.apply_async(MSPL.MergeSort, [A[mid:]])
        L = L.get()
        R = R.get()
    return Merge(L,R)

In [18]:
np.random.seed = 21521109

### Time to mesuare time of two method
first, we try with array have len is 100,000

In [19]:

arr = np.random.randint(0,2000000, size=100000)
MSPV0_time = Mesuare(arr, MergeSortParallelVer0)
MS_time = Mesuare(arr, MergeSort)
print(f"Parrallel version faster {float(MS_time)/MSPV0_time} times")

MergeSortParallelVer0, time used 820.2168941497803 (ms) 

MergeSort, time used 940.9980773925781 (ms) 

Parrallel version faster 1.1472551761665395 times


Seem not effective

But, we need to know that cost for create and manager process not small 

Let's try with array has len is 1,000,000

In [20]:
arr = np.random.randint(0,2000000, size=1000000)
MSPV0_time = Mesuare(arr, MergeSortParallelVer0)
MS_time = Mesuare(arr, MergeSort)
print(f"Parrallel version faster {float(MS_time)/MSPV0_time} times")

MergeSortParallelVer0, time used 6433.24613571167 (ms) 

MergeSort, time used 11197.022199630737 (ms) 

Parrallel version faster 1.740493362670334 times


We just boost time to complete merge sort x1.74 time not so good, but i belive that if len go to infinity (we all wish that not happen) Parallel version will also go to 2x times faster than normal version.

---

But 2 is k upper bound?

The answer is not i don't know upper bound of k in general and no one know, but, the computer know upper bound of k.

we know that k is the number process to sort k subarray, so, k is number of process that computer has.

method:
```
multiprocessing.cpu_count()
```
will return number of process that computer has, and you can use all of them (or not, it's just a maximize number of process, maybe sort-process must wait for another process like your system process, kernel, or fifty tab of chrome you are opening).

---

Below implementation is MergeSort Parellel will use maximize available process of your computer to work

## MergeSortParallel

In [21]:

def MergeSortParallel(A):
    n = len(A)
    if n <=1: 
        return A
    n_process = multiprocessing.cpu_count()

    A = np.array_split(A, n_process)
    
    pool = multiprocessing.Pool(n_process)
    A = pool.map(MSPL.MergeSort, A)
    while len(A)>1:
        odd_part = []
        if len(A) % 2 != 0:
            odd_part = A.pop()
        A = [(A[i],A[i+1]) for i in range(0,len(A),2)]
        if len(odd_part) == 0:
            A = pool.map(MSPL.Merge, A)
        else:
            A = pool.map(MSPL.Merge, A) + odd_part
    return A[0]

# Now let's mesuare again

In [22]:
MSP_time = Mesuare(arr, MergeSortParallel)
if (MS_time > MSP_time):
    print(f"MergeSort: Parallel version !!! FASTER !!! than normal version {MS_time/MSP_time} times!!!")
else:
    print(f"MergeSort: Parallel version !!! SLOWER !!! than normal version {MSP_time/MS_time} times!!!")

MergeSortParallel, time used 4734.275102615356 (ms) 

MergeSort: Parallel version !!! FASTER !!! than normal version 2.3650974979137067 times!!!


We got $\approx$ x2.36 boost with new version 

My computer has 8 process, but as i mentioned above, cost for create and manager process not small so we was not got x8 boost in this case. 

### Let's try with array has len is 2,000,000

In [23]:
arr = np.random.randint(0,2000000, size=2000000)
MSP_time = Mesuare(arr, MergeSortParallel)
MS_time = Mesuare(arr, MergeSort)
print(f"Parrallel version faster {float(MS_time)/MSP_time} times")

MergeSortParallel, time used 9430.230140686035 (ms) 

MergeSort, time used 23615.66138267517 (ms) 

Parrallel version faster 2.504250800920238 times


Now i got $\approx$ x2.5 times faster than normal mersort.

### In this case (my computer has 8 process) 

Again i belive that if len go to infinity (we all wish that not happen) Parallel version will also go to 8x times faster than normal version if all 8 process is for this work.


# The magical thing of this approach

What if each sub-array above i use quick sort instead of merge sort?

Let's try this.

In [24]:
import random

def QuickSort(arr):
    if len(arr) <= 1:
        return arr
    
    pivot = random.choice(arr)
    left = []
    right = []
    pivot_list = []
    
    for i in arr:
        if i < pivot:
            left.append(i)
        elif i > pivot:
            right.append(i)
        else:
            pivot_list.append(i)
    
    return QuickSort(left) + pivot_list + QuickSort(right)

In [30]:
QS_Time = Mesuare(arr, QuickSort)

QuickSort, time used 6832.568645477295 (ms) 



In [26]:

def MergeQuickSortParallel(A):
    n = len(A)
    if n <=1: 
        return A
    n_process = multiprocessing.cpu_count()

    A = np.array_split(A, n_process)
    
    pool = multiprocessing.Pool(n_process)
    A = pool.map(MSPL.QuickSort, A)
    while len(A)>1:
        odd_part = []
        if len(A) % 2 != 0:
            odd_part = A.pop()
        A = [(A[i],A[i+1]) for i in range(0,len(A),2)]
        if len(odd_part) == 0:
            A = pool.map(MSPL.Merge, A)
        else:
            A = pool.map(MSPL.Merge, A) + odd_part
    return A[0]

In [31]:
MQSP_Time = Mesuare(arr, MergeQuickSortParallel)
print(f"Merge Quick Sort faster {float(MSP_time)/MQSP_Time} times than old parallel version")
print(f"Quick Sort faster {float(MQSP_Time)/QS_Time} times than merge quick parallel version")

MergeQuickSortParallel, time used 9232.069969177246 (ms) 

Merge Quick Sort faster 1.0214643273036685 times than old parallel version
Quick Sort faster 1.3511858348160557 times than merge quick parallel version


- Seem efficiency not increase to clearly 

- and it even not good as simple quicksort version

- What happen?

- As i mention above cost for create and manager process is not small.

### So let's try with just 2 process

In [32]:
def MergeQuickSortParallelVer2(A):
    if (len(A) <=1): 
        return A
    mid = int(len(A)/2)
    with multiprocessing.Pool(2) as pool:
        L = pool.apply_async(MSPL.QuickSort, [A[0:mid]])
        R = pool.apply_async(MSPL.QuickSort, [A[mid:]])
        L = L.get()
        R = R.get()
    return Merge(L,R)

In [34]:
MQSP2_Time = Mesuare(arr, MergeQuickSortParallelVer2)
print(f"Merge Quick Sort ver 2 faster {float(MSP_time)/MQSP2_Time} times than old parallel version")
print(f"Quick Sort faster {float(MQSP2_Time)/QS_Time} times than merge quick parallel version 2")

MergeQuickSortParallelVer2, time used 6529.356956481934 (ms) 

Merge Quick Sort ver 2 faster 1.4442816043813163 times than old parallel version
Quick Sort faster 0.9556225916301525 times than merge quick parallel version 2


# Wow!
We just boost x1.4 times for old parallel version

And it's now faster than normal quick sort, althougt i use quick sort in this version but it still a good signal.

Seem we reach our target from begin,

# Funny section

Let's compare with np.sort method

In [36]:
builtin = Mesuare(arr, np.sort)
print(f"np.sort faster {float(MQSP2_Time)/builtin} times than merge quick sort parallel version 2")

sort, time used 112.08558082580566 (ms) 

np.sort faster 58.25331776287381 times than merge quick sort parallel version 2


Not too suprised, but im also a bit disappointed, I took the time to research that!!!